In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_chuko_mansions():
    base_url = 'https://www.homes.co.jp/mansion/chuko/tokyo/23ku/list/'
    max_page = 5 # 最大ページ数
    mansions = []

    for page in range(1, max_page):
        url = f'{base_url}?page={page}'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        for mansion in soup.select('.moduleInner'):
            try:
                name = mansion.select_one('.bukkenName').text.strip()

                # 所在地を改行の上部分と下部分に分けて取得
                location_parts = mansion.select('td')[0].text.strip()

                # 価格を取得
                price_text = mansion.select_one('.priceLabel').text.strip()
                price = int(price_text.replace('万円', '').replace(',', '')) * 10000

                # 間取りと専有面積を分けて取得
                layout_and_area = mansion.select_one('.layoutSpace').text.strip()
                layout, area = layout_and_area.split('/')

                mansions.append({
                    'name': name,
                    'location_parts': location_parts,
                    'price': price,
                    'layout': layout.strip(),
                    'area': area.strip()
                })
            except AttributeError:
                continue

    df = pd.DataFrame(mansions)
    return df

# マンション情報を取得
mansion_data = scrape_chuko_mansions()
print(mansion_data)


                 name                         location_parts      price  \
0           カーサフェリス洗足         東急目黒線 洗足駅 徒歩10分東京都品川区荏原7丁目9-16   79900000   
1    ライオンズマンション西麻布シティ     東京メトロ日比谷線 六本木駅 徒歩9分東京都港区西麻布2丁目13-8  113900000   
2         イニシアイオ新宿夏目坂       都営大江戸線 若松河田駅 徒歩6分東京都新宿区原町3丁目64-3   95800000   
3         クレストコート杉並宮前     京王井の頭線 富士見ヶ丘駅 徒歩11分東京都杉並区宮前1丁目15-1   74800000   
4        パークハウス高円寺南桃園  東京メトロ丸ノ内線 東高円寺駅 徒歩8分東京都杉並区高円寺南5丁目8-12   82900000   
..                ...                                    ...        ...   
115   コンシェリア馬込TERRACE       都営浅草線 西馬込駅 徒歩9分東京都大田区中馬込3丁目17-16   39800000   
116      グランイーグル大森南IV         東京モノレール 昭和島駅 徒歩13分東京都大田区大森南4丁目   39990000   
117          ベルジェンド王子     東京メトロ南北線 王子神谷駅 徒歩12分東京都北区豊島7丁目15-3   43990000   
118    京成本線 青砥駅 徒歩14分              京成本線 青砥駅 徒歩14分東京都葛飾区青戸7丁目   17500000   
119       プレシスヴィアラ鵜の木         東急多摩川線 鵜の木駅 徒歩4分東京都大田区鵜の木2丁目26   53500000   

    layout     area  
0     3LDK  60.32m²  
1     2LDK  56.46m²  
2     2LDK  59.13m²  
3     3LDK 

In [6]:
# 正規表現を使用して情報を抽出
mansion_data["transportation"] = mansion_data["location_parts"].str.extract(r"(.*) 徒歩(\d+)分")[0]
mansion_data["minutes"] = mansion_data["location_parts"].str.extract(r"徒歩(\d+)分")[0]
mansion_data["address"] = mansion_data["location_parts"].str.extract(r"徒歩\d+分(.*)")[0]


In [7]:
mansion_data

,name,location_parts,price,layout,area,transportation,minutes,address
0,カーサフェリス洗足,東急目黒線 洗足駅 徒歩10分東京都品川区荏原7丁目9-16,79900000,3LDK,60.32m²,東急目黒線 洗足駅,10,東京都品川区荏原7丁目9-16
1,ライオンズマンション西麻布シティ,東京メトロ日比谷線 六本木駅 徒歩9分東京都港区西麻布2丁目13-8,113900000,2LDK,56.46m²,東京メトロ日比谷線 六本木駅,9,東京都港区西麻布2丁目13-8
2,イニシアイオ新宿夏目坂,都営大江戸線 若松河田駅 徒歩6分東京都新宿区原町3丁目64-3,95800000,2LDK,59.13m²,都営大江戸線 若松河田駅,6,東京都新宿区原町3丁目64-3
3,クレストコート杉並宮前,京王井の頭線 富士見ヶ丘駅 徒歩11分東京都杉並区宮前1丁目15-1,74800000,3LDK,75.6m²,京王井の頭線 富士見ヶ丘駅,11,東京都杉並区宮前1丁目15-1
4,パークハウス高円寺南桃園,東京メトロ丸ノ内線 東高円寺駅 徒歩8分東京都杉並区高円寺南5丁目8-12,82900000,3LDK,72.26m²,東京メトロ丸ノ内線 東高円寺駅,8,東京都杉並区高円寺南5丁目8-12
...,...,...,...,...,...,...,...,...
115,コンシェリア馬込TERRACE,都営浅草線 西馬込駅 徒歩9分東京都大田区中馬込3丁目17-16,39800000,1LDK,41.12m²,都営浅草線 西馬込駅,9,東京都大田区中馬込3丁目17-16
116,グランイーグル大森南IV,東京モノレール 昭和島駅 徒歩13分東京都大田区大森南4丁目,39990000,3LDK,60.6m²,東京モノレール 昭和島駅,13,東京都大田区大森南4丁目
117,ベルジェンド王子,東京メトロ南北線 王子神谷駅 徒歩12分東京都北区豊島7丁目15-3,43990000,2LDK,49.43m²,東京メトロ南北線 王子神谷駅,12,東京都北区豊島7丁目15-3
118,京成本線 青砥駅 徒歩14分,京成本線 青砥駅 徒歩14分東京都葛飾区青戸7丁目,17500000,1K,20.72m²,京成本線 青砥駅,14,東京都葛飾区青戸7丁目
